<h1/>Задачи. Модуль "Динамическое программирование"</h1>

In [ ]:
import sys


def LIS_BottomUp_division(A, n):
    D = [1] * n
    for i in range(n):
        for j in range(i):
            if A[i] % A[j] == 0 and D[j] + 1 > D[i]:  # первое условие гарантирует A[i] => A[j]
                D[i] = D[j] + 1
    return max(D)


def main():
    reader = (map(int, line.split()) for line in sys.stdin)
    [n], A = next(reader), list(next(reader))
    print(LIS_BottomUp_division(A, n))


if __name__ == '__main__':
    main()

In [ ]:
import sys


def LnIS_with_binsearch(A, n):
    ends = [-1] * (n + 1)  # на элементе с каким индексом оканчивается оптимальная подпоследовательность длины i
    prev = [-1] * n  # индекс того элемента, который должен стоять перед A[i] в оптимальной подпоследовательности

    inf = -10 ** 10  # -бесконечность
    D = [inf] * (n + 1)
    D[0] = -inf      # +бесконечность

    for i in range(n):
        # реализуем двоичный поиск >A[i] в массиве D
        left = 0
        right = n
        while right - left > 1:
            middle = (left + right) // 2
            if D[middle] >= A[i]:
                left = middle
                prev[i] += left
            else:
                right = middle

        D[right] = A[i]
        ends[right] = i
        prev[i] = ends[right - 1]

    # восстанавливаем ответ
    result = []
    longest = None
    for k in range(n, -1, -1):
        if D[k] != inf:
            longest = k
            break
    t = ends[longest]
    while t != -1:
        result.append(t + 1)
        t = prev[t]
    result.reverse()

    return longest, result


def main():
    reader = (map(int, line.split()) for line in sys.stdin)
    [n], A = next(reader), list(next(reader))
    longest, indexes = LnIS_with_binsearch(A, n)
    print(longest)
    print(*indexes)


if __name__ == '__main__':
    main()

In [ ]:
import sys


def Levenshtein_distance(a, b):
    """Calculates the Levenshtein distance between a and b."""
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a, b = b, a
        n, m = m, n

    current_row = list(range(n+1)) # Keep current and previous row, not entire matrix
    for i in range(1, m+1):
        previous_row, current_row = current_row, [i]+[0]*n
        for j in range(1,n+1):
            add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
            if a[j-1] != b[i-1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]


def main():
    reader = (line.rstrip() for line in sys.stdin)
    a, b = next(reader), next(reader)
    print(Levenshtein_distance(a, b))


if __name__ == '__main__':
    main()

In [ ]:
import sys


def KnapSackWithoutRepsBU(capacity, n, weights):
    D = [[0] * (capacity + 1) for i in range(n + 1)]
    '''
    двумерный массив D[w, i] - максимальная стоимость рюкзака
    вместимости w, если разрешено использовать предметы 1...i
    '''
    for i in range(1, n + 1):
        for w in range(1, capacity + 1):
            if w >= weights[i - 1]:
                D[i][w] = max(D[i - 1][w], \
                              D[i - 1][w - weights[i - 1]] + weights[i -1])
            else:
                D[i][w] = D[i - 1][w]

    return D[n][capacity]


def main():
    reader = (map(int, line.split()) for line in sys.stdin)
    capacity, n = next(reader)    # вместимость рюкзака, количество слитков
    weights = list(next(reader))  # веса слитков
    assert len(weights) == n
    print(KnapSackWithoutRepsBU(capacity, n, weights))


if __name__ == '__main__':
    main()

In [ ]:
# Решение 1
import sys


def MaxStairsBU(n, numbers):
    D = [0] * (n + 1)
    D[1] = numbers[0]
    for i in range(2, n + 1):
        D[i] = numbers[i - 1] + max(D[i - 1], D[i - 2])
    return D[n]


def main():
    reader = (map(int, line.split()) for line in sys.stdin)
    [n], numbers = next(reader), list(next(reader))
    assert len(numbers) == n
    print(MaxStairsBU(n, numbers))


if __name__ == '__main__':
    main()

In [ ]:
# Решение 2 - используем меньше памяти
import sys


def MaxStairsBU(n, numbers):
    numbers = [0] + numbers
    for i in range(2, n + 1):
        numbers[i] += max(numbers[i-1], numbers[i-2])
    return numbers[n]


def main():
    reader = (map(int, line.split()) for line in sys.stdin)
    [n], numbers = next(reader), list(next(reader))
    assert len(numbers) == n
    print(MaxStairsBU(n, numbers))


if __name__ == '__main__':
    main()

In [ ]:
import sys


def CalculatorBU(n):
    '''
    Правильное решение заключается в нахождении для каждого числа от 2 до N
    минимального количества действий на основе предыдущих элементов, иначе
    говоря: F(N) = min(F(N-1), F(N/2), F(N/3)) + 1
    '''
    D = [0] * (n + 1)
    for i in range(2, n + 1):
        minimum = D[i - 1] + 1
        if i % 2 == 0: minimum = min(minimum, D[i // 2] + 1)
        if i % 3 == 0: minimum = min(minimum, D[i // 3] + 1)
        D[i] = minimum

    # Восстановим промежуточные числа
    solution = [n]
    k = n
    while k > 1:
        if D[k] == D[k - 1] + 1:
            solution.append(k - 1)
            k -= 1
            continue
        if k % 2 == 0 and D[k] == D[k // 2] + 1:
            solution.append(k // 2)
            k //= 2
            continue
        solution.append(k // 3)
        k //= 3
    solution.reverse()

    return D[n], solution


def main():
    n = int(input())
    res, solution = CalculatorBU(n)
    print(res)
    print(*solution)


if __name__ == '__main__':
    main()

<h1>Практика. Модуль "Динамическое программирование"</h1>
<h2>Задача "Расстояние редактирования"</h2>

<h3>Рекурсивный алгоритм:</h3>

In [ ]:
import sys
import random
from functools import lru_cache


def edit_distance(s1, s2):
    @lru_cache(maxsize=None)
    def d(i, j):
        if i == 0 or j ==0:
            return max(i, j)
        else:
            return min(d(i, j - 1) + 1,
                       d(i - 1, j) + 1,
                       d(i - 1, j - 1) + (s1[i - 1] != s2[j - 1]))

    return d(len(s1), len(s2))


def main():
    s1 = input()
    s2 = input()
    print(edit_distance(s1, s2))


def test(n_iter = 100):
    for i in range(n_iter):
        length = random.randint(0, 1000)
        s = "".join(random.choice("01") for _ in range(length))

        assert edit_distance(s, "") == edit_distance("", s) == len(s)
        assert edit_distance(s, s) == 0

    assert edit_distance("short", "ports") == 3
    # short-
    # p-orts


if __name__ == "__main__":
    # sys.getrecursionlimit() - проверит допустимую глубину рекурсии
    # установим лимит на 10000, но нам это не поможет - 10^6 операций
    sys.setrecursionlimit(10000)
    test()

<h3>Итеративный алгоритм:</h3>

In [ ]:
import random


def edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    d = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        d[i][0] = i
    for j in range(n + 1):
        d[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            d[i][j] = min(d[i][j - 1] + 1,
                          d[i - 1][j] + 1,
                          d[i - 1][j - 1] + (s1[i - 1] != s2[j - 1]))

    return d[m][n]


def main():
    s1 = input()
    s2 = input()
    print(edit_distance(s1, s2))


def test(n_iter = 100):
    for i in range(n_iter):
        length = random.randint(0, 64)
        s = "".join(random.choice("01") for _ in range(length))

        assert edit_distance(s, "") == edit_distance("", s) == len(s)
        assert edit_distance(s, s) == 0

    assert edit_distance("short", "ports") == 3
    # short-
    # p-orts


if __name__ == "__main__":
    test()

<h3>Оптимизация памяти, улучшение кода:</h3>

In [ ]:
import random


def edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    if m < n:
        return edit_distance(s2, s1)  # чтобы выбрать минимум памяти

    prev = list(range(n + 1))
    for i, ch1 in enumerate(s1, 1):
        curr = [i]  # O(min(m, n)) - памяти
        for j, ch2 in enumerate(s2, 1):
            curr.append(min(curr[-1] + 1,
                            prev[j] + 1,
                            prev[j - 1] + (ch1 != ch2)))
        prev = curr

    return prev[n]


def main():
    s1 = input()
    s2 = input()
    print(edit_distance(s1, s2))


def test(n_iter = 100):
    for i in range(n_iter):
        length = random.randint(0, 64)
        s = "".join(random.choice("01") for _ in range(length))

        assert edit_distance(s, "") == edit_distance("", s) == len(s)
        assert edit_distance(s, s) == 0

    assert edit_distance("short", "ports") == 3
    # short-
    # p-orts


if __name__ == "__main__":
    test()